# Proxy Pattern

Provides a surrogate or placeholder for another object to control access to it.

## Intent

- Control access to the original object
- Add functionality when accessing the object
- Delay the creation and initialization of expensive objects until needed
- Implement access control for the original object's clients

## Implementation 1: Basic Proxy Pattern

In [1]:
from abc import ABC, abstractmethod


class Subject(ABC):
    """Interface implemented by both RealSubject and Proxy"""

    @abstractmethod
    def request(self) -> None:
        pass


class RealSubject(Subject):
    """The real object that the proxy represents"""

    def request(self) -> None:
        print("RealSubject: Handling request.")


class Proxy(Subject):
    """The Proxy maintains a reference to the RealSubject and controls access to it"""

    def __init__(self, real_subject: RealSubject) -> None:
        self._real_subject = real_subject

    def check_access(self) -> bool:
        print("Proxy: Checking access prior to firing a real request.")
        return True

    def log_access(self) -> None:
        print("Proxy: Logging the request.")

    def request(self) -> None:
        """The proxy adds functionality before and after delegating to the real subject"""
        if self.check_access():
            self._real_subject.request()
            self.log_access()

### Usage

In [2]:
def client_code(subject: Subject) -> None:
    """Client code that works with all objects (both the real and proxy ones)"""
    subject.request()


if __name__ == "__main__":
    print("Client: Executing the client code with a real subject:")
    real_subject = RealSubject()
    client_code(real_subject)

    print("\nClient: Executing the same client code with a proxy:")
    proxy = Proxy(real_subject)
    client_code(proxy)

Client: Executing the client code with a real subject:
RealSubject: Handling request.

Client: Executing the same client code with a proxy:
Proxy: Checking access prior to firing a real request.
RealSubject: Handling request.
Proxy: Logging the request.


## Implementation 2: Protection (Access Control) Proxy

In [3]:
class SensitiveInfo:
    """Contains sensitive information that needs protection"""

    def __init__(self):
        self.users = {"admin": "admin_password", "user": "user_password"}

    def read(self):
        print("Reading sensitive information: ", self.users)

    def add(self, user, password):
        self.users[user] = password
        print(f"Added user: {user}")


class SensitiveInfoProxy:
    """Acts as a protection proxy for SensitiveInfo"""

    def __init__(self, username, password):
        self.sensitive_info = SensitiveInfo()
        self.username = username
        self.password = password

    def read(self):
        if self._authenticate():
            if self._authorize_read():
                self.sensitive_info.read()
            else:
                print("Authorization failed: Cannot read sensitive info")
        else:
            print("Authentication failed")

    def add(self, user, password):
        if self._authenticate():
            if self._authorize_write():
                self.sensitive_info.add(user, password)
            else:
                print("Authorization failed: Cannot modify sensitive info")
        else:
            print("Authentication failed")

    def _authenticate(self):
        stored_password = self.sensitive_info.users.get(self.username)
        return stored_password == self.password

    def _authorize_read(self):
        # All authenticated users can read
        return True

    def _authorize_write(self):
        # Only admin can modify
        return self.username == "admin"

### Usage

In [4]:
if __name__ == "__main__":
    # Admin can both read and write
    print("Admin operations:")
    admin_proxy = SensitiveInfoProxy("admin", "admin_password")
    admin_proxy.read()
    admin_proxy.add("new_user", "new_password")

    print("\nRegular user operations:")
    # Regular user can read but not write
    user_proxy = SensitiveInfoProxy("user", "user_password")
    user_proxy.read()
    user_proxy.add("another_user", "password123")  # Will fail

    print("\nInvalid credentials:")
    # Invalid credentials
    invalid_proxy = SensitiveInfoProxy("user", "wrong_password")
    invalid_proxy.read()  # Will fail

Admin operations:
Reading sensitive information:  {'admin': 'admin_password', 'user': 'user_password'}
Added user: new_user

Regular user operations:
Reading sensitive information:  {'admin': 'admin_password', 'user': 'user_password'}
Authorization failed: Cannot modify sensitive info

Invalid credentials:
Authentication failed


## Implementation 3: Virtual Proxy (Lazy Loading)

In [5]:
class ExpensiveObject:
    """Simulates an expensive object to create"""

    def __init__(self):
        self._heavy_data = None
        self._initialize()

    def _initialize(self):
        import time

        print("Creating expensive object...")
        time.sleep(2)  # Simulate heavy initialization
        self._heavy_data = [i for i in range(10000)]
        print("Expensive object created")

    def process(self):
        print(f"Processing data with {len(self._heavy_data)} items...")


class VirtualProxy:
    """Proxy that delays creation of the expensive object until needed"""

    def __init__(self):
        self._real_object = None

    def process(self):
        # Create the object only when needed
        if self._real_object is None:
            self._real_object = ExpensiveObject()
        self._real_object.process()

### Usage

In [6]:
if __name__ == "__main__":
    print("Creating proxy object (no expensive object created yet)...")
    proxy = VirtualProxy()
    print("Proxy created, but expensive object not created yet.")

    print("\nCalling process for the first time:")
    proxy.process()  # This will create the expensive object

    print("\nCalling process again (expensive object already created):")
    proxy.process()  # This will reuse the existing expensive object

Creating proxy object (no expensive object created yet)...
Proxy created, but expensive object not created yet.

Calling process for the first time:
Creating expensive object...
Expensive object created
Processing data with 10000 items...

Calling process again (expensive object already created):
Processing data with 10000 items...
